# Foundry Agent MCP test

### Setup Azure credentials and project 
1. use az cli to login to the tenant with your credential

<!-- initializing Project Client -->

In [12]:
from dotenv import load_dotenv

# load environment variables from .env file
load_dotenv(dotenv_path=".env", override=True)

from utils.fdyauth import AuthHelper
settings = AuthHelper.load_settings()
credential = AuthHelper.test_credential()

if credential:
    print('Environment and authentication OK')
else:
    print("please login first")

Environment and authentication OK


In [13]:
import os
import azure.ai.agents as agentslib
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    AgentEvaluationRequest,
    InputDataset,
    EvaluatorIds,
    EvaluatorConfiguration,
    AgentEvaluationSamplingConfiguration,
    AgentEvaluationRedactionConfiguration,
    Evaluation,
    DatasetVersion,
    FileDatasetVersion,
)
from azure.ai.agents.models import (
    FunctionTool,
    ToolSet,
    MessageRole,
)

# Import your custom functions to be used as Tools for the Agent
from utils.user_functions import user_functions

# Initialize project client with proper authentication
project_client = AIProjectClient(
    credential=credential,  # Use the credential from earlier setup
    endpoint=settings.project_endpoint
)
print(f"project client api version: {project_client._config.api_version}")
print(f"azure-ai-agents version: {agentslib.__version__}")

# AGENT_NAME = "Seattle Tourist Assistant"
# AGENT_INSTRUCTIONS = """You are a helpful tourist assistant"""

# # Add Tools to be used by Agent
# functions = FunctionTool(user_functions)

# toolset = ToolSet()
# toolset.add(functions)

# # To enable tool calls executed automatically
# project_client.agents.enable_auto_function_calls(tools=toolset, max_retry=4)

project client api version: 2025-05-15-preview
azure-ai-agents version: 1.1.0b3


In [14]:
AGENT_NAME = "product agent"
AGENT_INSTRUCTIONS = """You are a helpful product assistant"""

# Add Tools to be used by Agent
functions = FunctionTool({})

toolset = ToolSet()
toolset.add(functions)
toolset.add({
    "type": "mcp",
    "server_label": "product_info_mcp",
    "server_url": "https://gitmcp.io/Azure/azure-rest-api-specs",
    "require_approval": "never"
})

# To enable tool calls executed automatically
project_client.agents.enable_auto_function_calls(tools=toolset, max_retry=4)

### Create an AI agent (Azure AI Agent Service)

code_interpreter, file_search, function, bing_grounding, fabric_dataagent, bing_custom_search, sharepoint_grounding, azure_ai_search, openapi, connected_agent, azure_function, knowledge, browser_automation, deep_research

In [15]:
found_agent = None
all_agents_list = project_client.agents.list_agents()
for a in all_agents_list:
    if a.name == AGENT_NAME:
        found_agent = a
        break

if found_agent:
    agent = project_client.agents.update_agent(
        agent_id=found_agent.id,
        model=settings.model_deployment_name,
        instructions=AGENT_INSTRUCTIONS,
        tools=[{
            "type": "mcp",
            "server_label": "product_info_mcp",
            "server_url": "https://gitmcp.io/Azure/azure-rest-api-specs",
            "require_approval": "never"
        }],
        tool_resources=None,
    )
    project_client.agents.enable_auto_function_calls(tools=toolset, max_retry=4) 
    print(f"reusing agent > {agent.name} (id: {agent.id})")
else:
    agent = project_client.agents.create_agent(
        model=settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
        tools=[{
            "type": "mcp",
            "server_label": "product_info_mcp",
            "server_url": "https://gitmcp.io/Azure/azure-rest-api-specs",
            "require_approval": "never"
        }],
        tool_resources=None,
    )
    print(f"Created agent '{AGENT_NAME}' \nID: {agent.id}")
    # print(f"Created agent '{AGENT_NAME}' with {len(functions._functions)} tools\nID: {agent.id}")

reusing agent > product agent (id: asst_Erx0dtg3F8VE2iyy5sLD55qr)


### Create Thread

In [16]:
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_2ZqXCnvIP7SzNeE90oKm20qb


In [17]:
user_prompt= "use mcp tool and what are available github docs?"

msg = project_client.agents.messages.create(
        thread_id=thread.id,
        role=MessageRole.USER,
        content=user_prompt
)

In [18]:
run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)

print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

print(f"Run ID: {run.id}")

Run finished with status: RunStatus.COMPLETED
Run ID: run_RnFEwyCEo0Ak3zKYiYmnW3S3
